In [44]:
import pandas as pd
import nltk
tknzr=nltk.tokenize.TreebankWordTokenizer()

In [45]:
dataset=pd.read_csv("train_2kmZucJ.csv")
test=pd.read_csv("test_oJQbWVk.csv")

In [46]:
train_original=dataset.copy()
test_original=test.copy()

In [47]:
dataset.head(2)

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,2,0,Finally a transparant silicon case ^^ Thanks t...


In [48]:
dataset=dataset.drop("id",axis=1)

In [49]:
dataset.head(2)

,label,tweet
0,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,0,Finally a transparant silicon case ^^ Thanks t...


In [50]:
test.head(2)

,id,tweet
0,7921,I hate the new #iphone upgrade. Won't let me d...
1,7922,currently shitting my fucking pants. #apple #i...


In [51]:
import numpy as np

In [52]:
dataset.describe()

,label
count,7920.000000
mean,0.255808
std,0.436342
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


In [53]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [54]:
for index,tweet in enumerate(dataset["tweet"][10:15]):
    print(index+1,".",tweet)
for index,tweet in enumerate(test["tweet"][10:15]):
    print(index+1,".",tweet)

1 . hey #apple when you make a new ipod dont make it a new color or .2inches thinner make it not crash every five fuckin minite.
2 . Ha! Not heavy machinery but it does what I need it to. @Apple really dropped the ball with that design. #drinkyourhaterade
3 . Contemplating giving in to the iPhone bandwagon simply because #Cellcom has no new Androids #depressing #IDontWantTo
4 . I just made another crazy purchase lol my theory is 'Work hard, Play Hard' Lol #iPad #Apple #Shopping http://instagr.am/p/NUyFHvyE7I/
5 . @shaqlockholmes @sam_louise1991 the battery is so painful! I charge it overnight and by lunchtime battery is dead! #HateOrange.
1 . Con mi buddy#edgar #buddy #friend #viviendo #taller #friendly #hug #sony #smile #sonrisa http://instagram.com/p/fasVZruzQI/
2 . #MyMomIsGreatBecause she always care about me! And do anything for me #FOLLOW #MothersDay #MustFollow mothersday #iPhone #tweet
3 . #kiss,,#romance,#hot,#movie,#iphone,#sex,#porn Lustful Mature Woman Is Getting Nailed Int

In [55]:
import re
def clean_text(text):
    #Filter to allow only alphabets
    text = re.sub(r'[^a-zA-Z\']', ' ', text)
    
    #Remove Unicode characters
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    
    #Convert to lowercase to maintain consistency
    text = text.lower()
       
    return text

In [56]:
dataset["clean_text"]=dataset.tweet.apply(lambda x: clean_text(x))
test["clean_text"]=test.tweet.apply(lambda x: clean_text(x))

In [57]:
STOP_WORDS = ['a', 'about', 'above', 'after', 'again', 'against', 'all', 'also', 'am', 'an', 'and',
              'any', 'are', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below',
              'between', 'both', 'but', 'by', 'can', "can't", 'cannot', 'com', 'could', "couldn't", 'did',
              "didn't", 'do', 'does', "doesn't", 'doing', "don't", 'down', 'during', 'each', 'else', 'ever',
              'few', 'for', 'from', 'further', 'get', 'had', "hadn't", 'has', "hasn't", 'have', "haven't", 'having',
              'he', "he'd", "he'll", "he's", 'her', 'here', "here's", 'hers', 'herself', 'him', 'himself', 'his', 'how',
              "how's", 'however', 'http', 'i', "i'd", "i'll", "i'm", "i've", 'if', 'in', 'into', 'is', "isn't", 'it',
              "it's", 'its', 'itself', 'just', 'k', "let's", 'like', 'me', 'more', 'most', "mustn't", 'my', 'myself',
              'no', 'nor', 'not', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'otherwise', 'ought', 'our', 'ours',
              'ourselves', 'out', 'over', 'own', 'r', 'same', 'shall', "shan't", 'she', "she'd", "she'll", "she's",
              'should', "shouldn't", 'since', 'so', 'some', 'such', 'than', 'that', "that's", 'the', 'their', 'theirs',
              'them', 'themselves', 'then', 'there', "there's", 'these', 'they', "they'd", "they'll", "they're",
              "they've", 'this', 'those', 'through', 'to', 'too', 'under', 'until', 'up', 'very', 'was', "wasn't",
              'we', "we'd", "we'll", "we're", "we've", 'were', "weren't", 'what', "what's", 'when', "when's", 'where',
              "where's", 'which', 'while', 'who', "who's", 'whom', 'why', "why's", 'with', "won't", 'would', "wouldn't",
              'www', 'you', "you'd", "you'll", "you're", "you've", 'your', 'yours', 'yourself', 'yourselves']

In [58]:
def gen_freq(text):
    #Will store the list of words
    word_list = []

    #Loop over all the tweets and extract words into word_list
    for tw_words in text.split():
        word_list.extend(tw_words)

    #Create word frequencies using word_list
    word_freq = pd.Series(word_list).value_counts()
    
    #Drop the stopwords during the frequency calculation
    word_freq = word_freq.drop(STOP_WORDS, errors='ignore')
    
    return word_freq

#Check whether a negation term is present in the text
def any_neg(words):
    for word in words:
        if word in ['n', 'no', 'non', 'not'] or re.search(r"\wn't", word):
            return 1
        else:
            return 0

#Check whether one of the 100 rare words is present in the text
def any_rare(words, rare_100):
    for word in words:
        if word in rare_100:
            return 1
        else:
            return 0

#Check whether prompt words are present
def is_question(words):
    for word in words:
        if word in ['when', 'what', 'how', 'why', 'who']:
            return 1
        else:
            return 0

In [59]:
word_freq=gen_freq(dataset.clean_text.str)
rare_100=word_freq[-100]
dataset.head(2)
word_freq_test=gen_freq(test.clean_text.str)
test.head(2)

,id,tweet,clean_text
0,7921,I hate the new #iphone upgrade. Won't let me d...,i hate the new iphone upgrade won't let me d...
1,7922,currently shitting my fucking pants. #apple #i...,currently shitting my fucking pants apple i...


In [60]:
dataset['word_count']=dataset.clean_text.str.split().apply(lambda x: len(x))
test['word_count']=test.clean_text.str.split().apply(lambda x: len(x))
#dataset['word_count'] = dataset.clean_text.str.split().apply(lambda x: len(x))

dataset['any_neg']=dataset.clean_text.str.split().apply(lambda x: any_neg(x))
test['any_neg']=test.clean_text.str.split().apply(lambda x: any_neg(x))
#dataset['any_neg'] = dataset.clean_text.str.split().apply(lambda x: any_neg(x))
dataset['is_question']=dataset.clean_text.str.split().apply(lambda x: is_question(x))
test['is_question']=test.clean_text.str.split().apply(lambda x: is_question(x))
#dataset['is_question'] = dataset.clean_text.str.split().apply(lambda x: is_question(x))
#dataset['any_rare']=dataset.clean_text.str.split().apply(lambda x: any_rare(x,rare_100))
#dataset['any_rare']=dataset.clean_text.str.split().apply(lambda x: any_rare(x,rare_100))
#dataset['char_count'] = dataset.clean_text.apply(lambda x: len(x))
dataset['char_count'] = dataset.clean_text.apply(lambda x: len(x))
test['char_count'] = test.clean_text.apply(lambda x: len(x))
#word_freq = gen_fre(dataset.clean_text.str)
#100 most rare words in the dataset
#rare_100 = word_freq[-100:]
#Number of words in a tweet

#Negation present or not

#Prompt present or not

#Any of the most 100 rare words present or not

#Character count of the tweet

In [61]:
gen_freq(dataset.clean_text.str)[:10]

iphone       4313
apple        2974
p            2784
instagram    2187
samsung      1441
twitter      1262
new          1160
https        1014
phone         998
sony          855
dtype: int64

In [70]:
test.head(3)

,id,tweet,clean_text,word_count,any_neg,is_question,char_count
0,7921,I hate the new #iphone upgrade. Won't let me d...,i hate the new iphone upgrade won't let me d...,14,0,0,77
1,7922,currently shitting my fucking pants. #apple #i...,currently shitting my fucking pants apple i...,16,0,0,115
2,7923,"I'd like to puts some CD-ROMS on my iPad, is t...",i'd like to puts some cd roms on my ipad is t...,21,0,0,104


In [71]:
#from sklearn.model_selection import train_test_split

#X = dataset[['word_count', 'any_neg', 'any_rare', 'char_count', 'is_question']]
#y = dataset.label

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=27)
from sklearn.model_selection import train_test_split
X=dataset[['word_count','any_neg','char_count','is_question']]
X_test_data=test[['word_count','any_neg','char_count','is_question']]
y=dataset.label
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.1,random_state=27)

In [81]:
from sklearn.naive_bayes  import GaussianNB
model=GaussianNB()
model=model.fit(X_train,y_train)
pred=model.predict(X_test)
pred_test_value=model.predict(X_test_data)
pred_test_value.dtype

dtype('int64')

In [82]:
from sklearn.metrics import accuracy_score
print("Accuracy:",accuracy_score(y_test,pred)*100,"%")

Accuracy: 76.76767676767676 %


In [83]:
submission=pd.read_csv("sample_submission_LnhVWA4.csv")

In [84]:
submission['id']=test['id']
submission['label']=pred_test_value


In [85]:
pd.DataFrame(submission, columns=['id','label']).to_csv('logistic.csv')